<a href="https://colab.research.google.com/github/tontantip/DADS7202-Deep_learning/blob/main/study_NeuralNetWork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>